In [4]:
import keras
import pandas as pd
import numpy as np
from hyperopt import STATUS_OK,Trials,hp,tpe,fmin
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature

In [5]:
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv",sep=",")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
train,test=train_test_split(data,test_size=.25,random_state=42)

In [7]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

In [8]:
train_X,valid_X,train_y,valid_y=train_test_split(train_x,train_y,test_size=.20,random_state=42)
signature=infer_signature(train_x,train_y)

In [19]:
def train_model(params,epchs,train_X,train_y,valid_X,valid_y,test_x,test_y):
    mean=np.mean(train_X,axis=0)
    var=np.var(train_X,axis=0)
    model =keras.Sequential(
        [
            keras.Input([train_X.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=params['lr'],
                                                 momentum=params['momentum']),
                                                 loss="mean_squared_error",
                                                 metrics=[keras.metrics.RootMeanSquaredError()])


    with mlflow.start_run(nested=True):
        model.fit(train_X,train_y,validation_data=(valid_X,valid_y),epochs=epchs,batch_size=3)
        eval_result=model.evaluate(valid_X,valid_y,batch_size=3)
        eval_mse=eval_result[1]

        mlflow.log_params(params)
        mlflow.log_metric("eval_mse",eval_mse)
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss":eval_mse,'status':STATUS_OK,'model':model}

In [13]:
def Objective(params):
    result= train_model(
            params,
            epchs=3,
            train_X=train_X,
            train_y=train_y,
            valid_X=valid_X,
            valid_y=valid_y,
            test_x=test_x,
            test_y=test_y)
    return result

In [14]:
space={
    'lr':hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
    'momentum':hp.uniform('momentum',0.0,1.0)
}

In [20]:
mlflow.set_experiment('/winequality')
with mlflow.start_run():
    trails=Trials()
    best=fmin(
        fn=Objective,
        space=space,
        algo=tpe.suggest,
        max_evals=2
        )
    
    best_run=sorted(trails.results,key=lambda x:x['loss'])[0]

    mlflow.log_params(best)
    mlflow.log_metric('mse',best_run['loss'])
    mlflow.tensorflow.log_model(best_run['model'],"model",signature=signature)

    print(f'hyperparamter is {best}')
    print(f"mse is {best_run['loss']}")



Epoch 1/3                                            

  1/980 ━━━━━━━━━━━━━━━━━━━━ 1:49:59 7s/step - loss: 27.5659 - root_mean_squared_error: 5.2503
  2/980 ━━━━━━━━━━━━━━━━━━━━ 30s 31ms/step - loss: 24.1357 - root_mean_squared_error: 4.9003
  3/980 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - loss: 27.8178 - root_mean_squared_error: 5.2440  
  5/980 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - loss: 27.5645 - root_mean_squared_error: 5.2303  
  4/980 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - loss: 28.1975 - root_mean_squared_error: 5.2871  
  7/980 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - loss: 26.2364 - root_mean_squared_error: 5.1037
  8/980 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - loss: 25.5575 - root_mean_squared_error: 5.0359
  6/980 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - loss: 26.7634 - root_mean_squared_error: 5.1037
 10/980 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 31.0705 - root_mean_squared_error: 5.6483
  9/980 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - loss: 28.2278 - root_mean_squared_error: 5.2588
 11/980

2024/10/21 18:08:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ADMIN\AppData\Local\Temp\tmpjubkuyau\model, flavor: tensorflow). Fall back to return ['tensorflow==2.17.0', 'cloudpickle==3.1.0']. Set logging level to DEBUG to see the full traceback. 



100%|██████████| 2/2 [05:49<00:00, 174.83s/trial, best loss: 0.7970542311668396]


IndexError: list index out of range